Lab2

In [ ]:
import pandas as pd
import urllib.request
import datetime
import os
import shutil
from IPython.display import display


Loading data and writing it to corresponding files

In [134]:
def get_url(i: int) -> str:
    return f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean'

def file_create(i: int, data: bytes, csv_path: str) -> None:
    date_time = datetime.datetime.now().strftime("%d%m%Y%H%M%S")
    out = open(f'{csv_path}/vhi_id_{i}_{date_time}.csv', "wb")
    out.write(data)

if os.path.isdir(csv_path:="csv_files"):
    shutil.rmtree(csv_path)

os.mkdir(csv_path)

for i in range(1, 28):
    url = get_url(i)
    data = urllib.request.urlopen(url).read()
    file_create(i, data, csv_path)
print("Files with data created")


Files with data created


Reading csv data into pandas dataframes

In [135]:
headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']

def dataframe_create(path: str):    
    for file in (os.listdir(path)):
        i = file.split(sep='_')[2]
        df = pd.read_csv(f'{path}/{file}', header = 1, names = headers)

        print(f'File {file} dataframe:')
        
        # dropping NaN values
        df = df.dropna(subset=['VHI']).drop(columns=['empty'])

        # normalizing data
        df["Year"] = df["Year"].str.extract(r"(\d+)").astype(int)
        df['Week'] = df['Week'].astype(int)
        df.insert(0, 'ProvinceID', [int(i)] * df.shape[0])
        
        display(df.head())

dataframe_create(csv_path)

File vhi_id_10_19022025193540.csv dataframe:


,ProvinceID,Year,Week,SMN,SMT,VCI,TCI,VHI
0,10,1982,1,0.059,258.24,51.11,48.78,49.95
1,10,1982,2,0.063,261.53,55.89,38.20,47.04
2,10,1982,3,0.063,263.45,57.30,32.69,44.99
3,10,1982,4,0.061,265.10,53.96,28.62,41.29
4,10,1982,5,0.058,266.42,46.87,28.57,37.72


File vhi_id_11_19022025193540.csv dataframe:


,ProvinceID,Year,Week,SMN,SMT,VCI,TCI,VHI
0,11,1982,1,0.077,262.50,61.42,34.11,47.76
1,11,1982,2,0.084,264.78,67.23,24.00,45.62
2,11,1982,3,0.086,265.88,67.65,20.41,44.03
3,11,1982,4,0.084,266.90,65.13,19.09,42.11
4,11,1982,5,0.078,266.89,55.58,25.02,40.30


File vhi_id_12_19022025193542.csv dataframe:


...

,ProvinceID,Year,Week,SMN,SMT,VCI,TCI,VHI
0,7,1982,1,0.070,255.79,27.63,78.38,53.00
1,7,1982,2,0.073,257.68,30.82,71.96,51.39
2,7,1982,3,0.077,259.75,33.88,64.97,49.42
3,7,1982,4,0.080,261.90,34.23,58.57,46.40
4,7,1982,5,0.082,263.84,32.05,53.87,42.96


File vhi_id_8_19022025193538.csv dataframe:


,ProvinceID,Year,Week,SMN,SMT,VCI,TCI,VHI
0,8,1982,1,0.038,252.22,47.05,69.15,58.10
1,8,1982,2,0.036,253.39,45.39,63.52,54.45
2,8,1982,3,0.033,254.22,41.22,60.22,50.72
3,8,1982,4,0.031,255.60,37.35,58.01,47.68
4,8,1982,5,0.030,257.16,32.54,56.92,44.73


File vhi_id_9_19022025193539.csv dataframe:


,ProvinceID,Year,Week,SMN,SMT,VCI,TCI,VHI
0,9,1982,1,0.079,267.92,43.33,29.71,36.52
1,9,1982,2,0.074,267.83,42.28,28.60,35.44
2,9,1982,3,0.076,267.98,44.17,28.60,36.38
3,9,1982,4,0.077,268.75,45.15,28.16,36.66
4,9,1982,5,0.077,269.44,44.22,28.30,36.26
